<a href="https://colab.research.google.com/github/gyeongdong13/Study/blob/main/CNN_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

keras 패키지를 활용한 CNN 구현 실습

In [ ]:
import tensorflow as tf          #딥러닝 패키지
import numpy as np               #배열계산 패키지
import matplotlib.pyplot as plt  #시각화 패키지

1. MNIST Data 불러오기

In [ ]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

print("train dataset shape:", train_images.shape, train_labels.shape)
print("test dataset shape:", test_images.shape, test_labels.shape)

11490434/11490434 [==============================] - 0s 0us/step
train dataset shape: (60000, 28, 28) (60000,)
test dataset shape: (10000, 28, 28) (10000,)


2. MNIST 데이터 전처리 수행

In [ ]:
# MNIST 데이터의 0~255 픽셀 값을 0에서 1 사이값으로 scale하고, 데이터 타입을 float32로 변경해준다.

# Keras 패키지에서는 입력데이터가 float 이어야 하기 때문.
# 또한, 일반적으로 딥러닝은 input 값에 대하여 스케일리 해주어야 한다.

# 이미지 값 변환
train_images = np.array(train_images/255.0, dtype=np.float32) # 이미지의 픽셀을 줄여주기 위해(라벨에서는 나누기 255 X)
test_images = np.array(test_images/255.0, dtype=np.float32)

# 라벨 값 변환
train_labels = np.array(train_labels, dtype=np.float32)
test_labels = np.array(test_labels, dtype=np.float32)

print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

# ANN은 입력층에 1차원 CNN은 입력층에 2차원으로 된다는 차이점

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [ ]:
# 라벨값을 원핫인코딩(One-Hot-Encoding) 해주어야 함

from tensorflow.keras.utils import to_categorical

train_labels_2 = to_categorical(train_labels)
test_labels_2 = to_categorical(test_labels)

train_labels_2

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [ ]:
# 학습데이터를 학습데이터와 검증데이터로 분리
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(train_images, train_labels_2, test_size=0.3, random_state=1)

train_x.shape, val_x.shape, train_y.shape, val_y.shape

((42000, 28, 28), (18000, 28, 28), (42000, 10), (18000, 10))

3. CNN 모델의 구조 설계

In [ ]:
# 필요 패키지 불러오기

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

In [ ]:
# 딥러닝 설계하는 두가지 방법(Keras 패키지)

# 순서대로 쌓는 방법 : Sequential() -> model.add()

# 기능형 방법 : 선언방식 -> input() -> CNN은 대부분 기능형

In [ ]:
# Keras Functional Coding 방법
# 계속 뒤에 이어주면서 작성할 수 있다,

# 기능형 방법
# filters : 특징추출 맵 32개 생성
# 커널 사이즈 : 필터 크기
# 스트라이드 : 몇칸씩 이동할지 선택
# 패딩 : "same" 특징추출이 끝났을 때 동일한 사이즈로 결과가 나오게끔 설정
# 활성화 함수 : relu

input_layer = Input(shape=(28, 28, 1)) # shape = 가로, 세로, 1(흑백을 의미)
conv_layer_1 = Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu')(input_layer) # kernel_size는 3이면 3*3을 의미
conv_layer_1 = Conv2D(filters=64, kernel_size=3, activation='relu')(conv_layer_1)
pooling_layer_1 = MaxPooling2D(2)(conv_layer_1) # 맥스풀링 2 : 크기를 2배만큼 압축해라

# 3차원으로 되어있는 Feature map 결과를 Fully Connected 연결하기 위해서는 Flatten()을 적용해야함.
FC_layer = Flatten()(pooling_layer_1) # 마지막 특징추출을 한 줄로 정리하는 레이어
FC_layer = Dense(100, activation='relu')(FC_layer)
output = Dense(10, activation='softmax')(FC_layer)

model = Model(inputs=input_layer, outputs=output) # 기능형 방법 마무리(CNN은 여기까지 해줘야 됨)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 64)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 10816)             0         
                                                                 
 dense (Dense)               (None, 100)               1081700   
                                                             

4. CNN 모델의 손실함수(Loss)와 최적화함수(Optimizer) 설정

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

5. CNN 모델 학습 수행 및 성능평가

In [ ]:
epochs = 2
batch_size = 256 #주로 2의 배수로 선정함

history = model.fit(x=train_x, y=train_y, batch_size=batch_size, validation_data=(val_x, val_y),
                    epochs=epochs, verbose=1)

Epoch 1/2
165/165 [==============================] - 146s 887ms/step - loss: 0.0560 - accuracy: 0.9832 - val_loss: 0.0645 - val_accuracy: 0.9798
Epoch 2/2
165/165 [==============================] - 141s 856ms/step - loss: 0.0370 - accuracy: 0.9889 - val_loss: 0.0667 - val_accuracy: 0.9799


In [ ]:
# 테스트 데이터 세트로 모델 성능 검증
# 딥러닝에서는 accuracy를 계산하는데 model.evaluate를 model.score 대신 사용함
# 첫번째 결과값은 loss, 두번째 결과값은 accuracy를 의미함

model.evaluate(test_images, test_labels_2, verbose=0)

In [ ]:
# 시각화를 통하여 학습이 잘 이루어지고 있는지 확인

import matplotlib.pyplot as plt

def show_history(history):
    plt.plot(history.history['accuracy'], label='train')
    plt.plot(history.history['val_accuracy'], label='valid')
    plt.legend()

show_history(history)

6. Dropout을 적용하여 Fully Connected Layer의 오버피팅 조정

In [ ]:
# CNN은 일반적으로 Dense Layer보다는 파라미터수 작음
# 하지만 많은 Filter 들을 적용하고 이를  Fully Connected Layer로 연결 시 파라미터 수가 늘어남
# Flatten() 이후 Dropout을 적용하여 특정 비율로 FC Layer 연결을 누락 적용
from tensorflow.keras.layers import Dropout

input_tensor = Input(shape=(28, 28, 1))
x = Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu')(input_tensor)
x = Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x = MaxPooling2D(2)(x)

x = Flatten()(x)
x = Dropout(rate=0.5)(x)   # 이 부분에 추가
x = Dense(100, activation='relu')(x)

output = Dense(10, activation='softmax')(x)

model = Model(inputs=input_tensor, outputs=output)

In [ ]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 5
batch_size = 128 #주로 2의 배수로 선정함

history = model.fit(x=train_x, y=train_y, batch_size=batch_size, validation_data=(val_x, val_y),
                    epochs=epochs, verbose=1)

model.evaluate(test_images, test_labels_2, verbose=0)